## PM Chatbot POC
### By Juan Fernando Lavieri Goiri

In [85]:
!pip install gradio pandas transformers openai

In [86]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [87]:
# Accessing the OpenAI API Key
from google.colab import userdata
OPEN_AI_TOKEN = userdata.get('OPEN_AI_TOKEN')

# Accessing the path to data
PATH_TO_DATA = userdata.get('PATH_TO_DATA')

# Accessing the HugginFace Token
from google.colab import userdata

In [88]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os
import requests
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from datetime import datetime
from pathlib import Path
import shutil
import openai
import re
import numpy as np
import gradio as gr
import logging

In [89]:
# Loading data
# This data was generated using https://www.mockaroo.com/
data = pd.read_csv(PATH_TO_DATA)
data['Created Date'] = pd.to_datetime(data['Created Date'])
data['Closed Date'] = pd.to_datetime(data['Closed Date'])
data['Activated Date'] = pd.to_datetime(data['Activated Date'])
data['Time to Close'] = (data['Closed Date'] - data['Activated Date']).dt.days
# Setting the explicit sprint start and end dates
sprint_start_date = pd.to_datetime('2024-04-29')
sprint_end_date = pd.to_datetime('2024-05-10')
sprint_days_explicit = (sprint_end_date - sprint_start_date).days + 1  # Including both start and end date

# Creating a date range for the sprint based on the provided dates
sprint_date_range_explicit = pd.date_range(start=sprint_start_date, end=sprint_end_date)


<ipython-input-89-ae02dc6ce850>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Created Date'] = pd.to_datetime(data['Created Date'])
<ipython-input-89-ae02dc6ce850>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Closed Date'] = pd.to_datetime(data['Closed Date'])
<ipython-input-89-ae02dc6ce850>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Activated Date'] = pd.to_datetime(data['Activated Date'])


In [90]:
data.head()

,ID,Work Item Type,Title,Assigned To,State,Tags,Story Points,Parent,Created Date,Closed Date,Activated Date,Time to Close
0,1,User Story,Triple-buffered bandwidth-monitored customer l...,Loki Odinson <thor@avengers.com>,Closed,Design,2,Chatbot,2024-04-16 21:57:45,2024-05-08 18:37:15,2024-05-05 17:21:22,3
1,2,Bug,Diverse bandwidth-monitored task-force,Anthony Stark <iamironman@avengers.com>,Closed,Frontend,5,Chatbot,2024-04-22 06:08:08,2024-05-04 06:33:55,2024-05-01 13:14:03,2
2,3,User Story,Enterprise-wide fault-tolerant system engine,Natasha Romanov <1wDf1!@avengers.com>,Closed,Frontend,2,Data Governance Automation,2024-04-24 10:06:31,2024-05-09 01:36:27,2024-05-04 10:24:52,4
3,4,User Story,Persevering static secured line,Anthony Stark <iamironman@avengers.com>,Closed,Backend,5,Chatbot,2024-04-18 06:01:07,2024-05-08 13:29:32,2024-05-06 21:07:31,1
4,5,Bug,Synergized hybrid emulation,Thor Odinson <the_real_thor@avengers.com>,Closed,Backend,2,Data Governance Automation,2024-04-17 08:54:59,2024-05-03 10:52:44,2024-05-06 03:38:26,-3


The dataset contains the following columns relevant to our analysis:

ID: Unique identifier for the work item.

Work Item Type: Type of the work item, e.g., User Story.

Title: Title of the work item.

Assigned To: The individual to whom the work item is assigned.

State: Current state of the work item (e.g., New, Closed).

Tags: Tags associated with the work item, indicating focus areas or categories.

Story Points: Effort estimate for the work item.

Parent: Identifier for the parent item, linking work items to broader initiatives or epics.

Created Date: Date and time when the work item was created.

Closed Date: Date and time when the work item was closed (if applicable).

Activated Date: Date and time when the work item was made active (if applicable).

#PM Chatbot

In [95]:
client = openai.OpenAI(
    api_key=OPEN_AI_TOKEN  # Ensure your API key is securely accessed via environment variables
)

# Initialize an empty list to store chat history
chat_history = []

# Configure logging
logging.basicConfig(level=logging.DEBUG)

def chat_with_model(query, history):
    global data
    global chat_history  # Access the global chat history list
    try:
        # Extracting the text of the user's query
        user_query_text = query['text']

        # Append the user's query to the chat history
        chat_history.append({"role": "user", "content": user_query_text})

        # Combine chat history with the current query
        chat_with_history = history + [{"role": "user", "content": user_query_text}]

        # Log the chat history with the current query
        logging.debug("Chat history with current query:")
        logging.debug(chat_with_history)

        # Create chat completion
        logging.debug("Sending messages to the chat model...")
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": f"Your name is P.M. (Project Multimodal)), You are the best project manager in the universe and you are specially good at doing sprint reports and providing suggestions based on {data}. You have access to the sprint data frame {data} and can analyze metrics like story points, task distribution, and sprint progress. Here is the sprint data frame: {data}. The user may ask any questions about the sprint data, and you'll provide the user with the analysis they need and perform any needed calculations based on the data {data}. You don't provide sample calculations but instead always provide answers based on the data frame: {data}, you perform all calculations the user request and always look to be as usefull as possible. You will make use of the chat history to keep context on what the user said on the previous messages a provide responses in context, here is the chat history {chat_history}", "type": "text"},
                {"role": "user", "content": user_query_text, "type": "text"}
            ]
        )

        # Extract the model's response from the completion
        message_content = response.choices[0].message.content

        # Log the model's response
        logging.debug("Model's response:")
        logging.debug(message_content)

        # Append the model's response to the chat history
        chat_history.append({"role": "system", "content": message_content})

        return message_content
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        return f"An error occurred: {str(e)}"






In [96]:
print(chat_history)

[]


In [97]:
# Setting up the Gradio interface
iface = gr.ChatInterface(fn=chat_with_model,
                     examples=[{"text": "How many total story points were completed?"}, {"text": "What was the distribution of work items by state?"}, {"text": "Who where the top contributors?"}, {"text": "Do a full sprint report with recomendations."},
                               {"text": "What was the work distribution by Parent?"}, {"text": "What was the work distribution by tag?"}, {"text": "Who was the most efficient contributor?"}],
                     title= "Sprint Data Chatbot",
                     multimodal=True)
# iface.launch(debug = True)
iface.launch()





Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9a328ff5de579eeaab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
